In [2]:
#from __future__ import unicode_literals, print_function, division
import numpy
import scipy
import matplotlib
import pandas
import statsmodels
import sklearn
import tensorflow
import keras
from io import open
import unicodedata
import string
import re
import random
import random
import math
import os
import time
import torch
import torch.nn as nn
from torch import optim
import spacy
import torch.nn.functional as F
from pickle import dump
from unicodedata import normalize
from numpy import array
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle
import string
import re

SEED = 1
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

INDIC_NLP_LIB_HOME=r"/home/arushi/Neural-Machine-Translation/anoopkunchukuttan-indic_nlp_library-eccde81/src"
INDIC_NLP_RESOURCES=r"/home/arushi/Neural-Machine-Translation/indic_nlp_resources-master"
import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
from indicnlp.tokenize import indic_tokenize
from pickle import dump
from unicodedata import normalize
from numpy import array

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Using TensorFlow backend.
/home/arushi/Neural-Machine-Translation/anoopkunchukuttan-indic_nlp_library-eccde81/src/indicnlp/script/indic_scripts.py:116: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  ALL_PHONETIC_VECTORS= ALL_PHONETIC_DATA.ix[:,PHONETIC_VECTOR_START_OFFSET:].as_matrix()
/home/arushi/Neural-Machine-Translation/anoopkunchukuttan-indic_nlp_library-eccde81/src/indicnlp/script/indic_scripts.py:117: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  TAMIL_PHONETIC_VECTORS=TAMIL_PHONETIC_DATA.ix[:,PHONETIC_VECTOR_START_OFFSET:].as_matrix()
/home/arushi/Neural-Machine-Translation/anoopkunchukuttan-indic_nlp_library-eccde81/src/indicnlp/script/english_script.py:113: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  ENGLISH_PHONETIC_VECTORS=ENGLISH_PHONETIC_DATA.ix[:,PHONETIC_VECTOR_START_OFFSET:].as_matrix()


In [4]:
SOS_token = 0
EOS_token = 1

In [5]:
class Lang:
    def __init__(self, name):
        self.name = name
        #index of each new word
        self.word2index = {}
        #count of that word
        self.word2count = {}
        #index with word
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

def load_doc(filename):
    file = open(filename, mode='rt', encoding='utf-8')
    text = file.read()
    file.close()
    return text

In [6]:
def to_pairs(english_text, hindi_text):
    english_lines = english_text.strip().split('\n')
    hindi_lines = hindi_text.strip().split('\n')
    pairs = []
    for i in range(len(hindi_lines)):
        pairs.append([])
        pairs[i].append(pre_process_english_sentence(english_lines[i]))
        pairs[i].append(pre_process_hindi_sentence(hindi_lines[i]))
    return pairs

In [7]:
def clean_text(line):
    text = line
    text=text.replace(u',','')
    text=text.replace(u'"','')
    text=text.replace(u'(','')
    text=text.replace(u')','')
    text=text.replace(u'"','')
    text=text.replace(u':','')
    text=text.replace(u"'",'')
    text=text.replace(u"‘‘",'')
    text=text.replace(u"’’",'')
    text=text.replace(u"''",'')
    text=text.replace(u".",'')
    text=text.replace(u"-",'')
    text=text.replace(u"।",'')
    text=text.replace(u"?",'')
    text=text.replace(u"\\",'')
    text=text.replace(u"_",'')
    text=re.sub('[a-zA-Z]', '', text)
    text=re.sub('[0-9+\-*/.%]', '', text)
    return text

def pre_process_english_sentence(line):
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    table = str.maketrans('', '', string.punctuation)
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    line = line.split()
    line = [word.lower() for word in line]
    line = [word.translate(table) for word in line]
    line = [re_print.sub('', w) for w in line]
    line = [word for word in line if word.isalpha()]
    #line.reverse()
    line = ' '.join(line)
    return line

def pre_process_hindi_sentence(line):
    remove_nuktas=False
    factory=IndicNormalizerFactory()
    normalizer=factory.get_normalizer("hi",remove_nuktas)
    line=normalizer.normalize(line)
    line=clean_text(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line):
        tokens.append(t)
    line = tokens
    line = [word.lower() for word in line]
    line = [word for word in line if not re.search(r'\d', word)]
    line = ' '.join(line)
    return (line)

In [8]:
def prepareData(pairs):
    input_lang = Lang('eng')
    output_lang = Lang('hin')
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    return input_lang, output_lang

def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

def max_length(lines):
    return max(len(line.split()) for line in lines)

In [9]:
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

In [10]:
english_text = load_doc('English')
hindi_text = load_doc('Hindi')
pairs = to_pairs(english_text, hindi_text)
pairs = array(pairs)
save_clean_data(pairs, 'english-hindi.pkl')
#input_lang, output_lang = prepareData(pairs)

Saved: english-hindi.pkl


In [13]:
#dataset size that to be taken
n_sentences = len(pairs)
dataset = pairs[:n_sentences, :]
n_training_samples = (int)(n_sentences/2)
train, test = dataset[:n_training_samples], dataset[n_training_samples:]
#shuffle(train)
#shuffle(test)
save_clean_data(dataset, 'english-hindi-both.pkl')
save_clean_data(train, 'english-hindi-train.pkl')
save_clean_data(test, 'english-hindi-test.pkl')

[['saliva has important role in keeping the mouth clean'
  'मुँह साफ रखने में लार की खास भूमिका होती है']
 ['eat only that food in which there is less fat and more fiber'
  'खाना वही खाएँ जिसमें कम वसा और फाइबर ज्यादा हो']
 ['but you can keep yourself healthy by paying attention to your food habits and by bringing a little change in your lifestyle'
  'लेकिन खानपान पर ध्यान और जीवनशैली में थोड़ा बदलाव लाकर आप खुद को स्वस्थ रख सकते हैं']
 ['keep doing light physical activities'
  'करते रहें हल्की फुल्की शारीरिक गतिविधियाँ']
 ['but one also gets relief from stinking breath'
  'बल्कि साँसों की दुर्गंध से भी छुटकारा मिलता है']
 ['get the teeth checkedup with the dentists regularly'
  'दंतचिकित्सक से दाँतों की जाँच नियमित रूप से कराएँ']
 ['it is not less than a challenge to keep oneself healthy in today s busy life'
  'आज की व्यस्त जिंदगी में खुद को स्वस्थ रखना किसी चुनौती से कम नहीं']
 ['bacteria stay between our gums and teeth'
  'हमारे मसूढ़ों और दाँतों के बीच बैक्टीरिया मौजूद होते हैं']


In [ ]:
dataset = load_clean_sentences('english-hindi-both.pkl')
train = load_clean_sentences('english-hindi-train.pkl')
test = load_clean_sentences('english-hindi-test.pkl')

In [ ]:
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare hindi tokenizer
hindi_tokenizer = create_tokenizer(dataset[:, 1])
hindi_vocab_size = len(hindi_tokenizer.word_index) + 1
hindi_length = max_length(dataset[:, 1])
print('German Vocabulary Size: %d' % hindi_vocab_size)
print('German Max Length: %d' % (hindi_length))

In [ ]:
# prepare training data
trainX = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_sequences(hindi_tokenizer, hindi_length, train[:, 1])
trainY = encode_output(trainY, hindi_vocab_size)
# prepare validation data
trainX = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
trainY = encode_sequences(hindi_tokenizer, hindi_length, test[:, 1])
trainY = encode_output(trainY, hindi_vocab_size)